In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import sys 
# 设置显示选项，使所有行和列都能完整显示
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# 然后执行你的代码，输出将不再被省略


In [2]:
def myprint(data):
    """
    Redirects the output of a function to a file.

    Args:
        filename (str): The name of the file to redirect the output to.
        func (function): The function whose output needs to be redirected.

    Returns:
        None
    """
    with open('output.txt', 'w') as f:
        sys.stdout = f
        print(data)
    sys.stdout = sys.__stdout__

# # 示例函数，将其输出重定向到文件
# def example_function(data):
#     print("data")

# 使用封装好的函数来重定向输出到文件
# redirect_output_to_file('output.txt')


In [2]:
def load_dataset(data_dir):
    mb = pd.read_csv(data_dir)
    
    # #META_TARGETS = ['DistComp', 'Recall', 'QueryTime']
    # META_TARGETS = ['Recall']
    #划分train,test
    #返回划分后的csv格式数据
    train,val=train_test_split(mb, test_size=0.25)

    # get_xy = lambda x: (x.drop(META_TARGETS, axis=1).copy(), x.loc[:, META_TARGETS].copy())
    # #将元数据库中图配置数据和元目标分离
    # train_x,train_y=get_xy(train)
    # test_x,test_y=get_xy(test)
    
    return train,val

def rand_train(train,batch_size):
    length=train.shape[0]
    
    ans=[]
    l=0
    r=min(batch_size,length)
    
    while l<length:
        ans.append(train.iloc[l:r])
        l=r
        r=min(r+batch_size,length)
    
    return ans

In [4]:
data_dir = '/home/sfy/study/myformer/data/train_nsw.csv'
mb_train,mb_val= load_dataset(data_dir)
print(mb_train)

            dataset   size  dim graph_type  NN  ef_construction   k    recall  \
4032   mnist6W_4000   4000  784        nsw   2               70   1  0.275000   
1959  crawl1M_10000  10000  300        nsw  15               90   1  0.760000   
953     sift1M_5000   5000  128        nsw  30              140  30  0.996000   
4490   mnist6W_8000   8000  784        nsw  15               50  30  0.986667   
4727  mnist6W_16000  16000  784        nsw   1               70  30  0.206667   
...             ...    ...  ...        ...  ..              ...  ..       ...   
2245   crawl1M_5000   5000  300        nsw   5               50  10  0.912000   
2795   crawl1M_1000   1000  300        nsw  40              120  30  1.000000   
2541   crawl1M_1000   1000  300        nsw   1               50   1  0.000000   
3596   mnist6W_1000   1000  784        nsw  50              150  30  1.000000   
1722  crawl1M_50000  50000  300        nsw  45               70   1  0.810000   

          qps  distcomp  
4

In [7]:
train=mb_train[['dataset','size','dim','graph_type','NN','ef_construction','k','recall']]
print(train)
train.graph_type = train.graph_type.apply(lambda x: 0 if x == 'nsw' else x)
print(train.size)

            dataset   size  dim graph_type  NN  ef_construction   k    recall
4032   mnist6W_4000   4000  784        nsw   2               70   1  0.275000
1959  crawl1M_10000  10000  300        nsw  15               90   1  0.760000
953     sift1M_5000   5000  128        nsw  30              140  30  0.996000
4490   mnist6W_8000   8000  784        nsw  15               50  30  0.986667
4727  mnist6W_16000  16000  784        nsw   1               70  30  0.206667
...             ...    ...  ...        ...  ..              ...  ..       ...
2245   crawl1M_5000   5000  300        nsw   5               50  10  0.912000
2795   crawl1M_1000   1000  300        nsw  40              120  30  1.000000
2541   crawl1M_1000   1000  300        nsw   1               50   1  0.000000
3596   mnist6W_1000   1000  784        nsw  50              150  30  1.000000
1722  crawl1M_50000  50000  300        nsw  45               70   1  0.810000

[4356 rows x 8 columns]
34848


/tmp/ipykernel_1784103/1133516515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.graph_type = train.graph_type.apply(lambda x: 0 if x == 'nsw' else x)


In [9]:
train.size = np.vectorize(lambda x: np.log10(x))(train.size)
print(train.size)

34848


/tmp/ipykernel_1784103/3394985218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.size = np.vectorize(lambda x: np.log10(x))(train.size)


In [28]:
for iter in rand_train(train,100):
    bases = iter.dataset.unique()
    # print("bases:",bases)
    for dataset in bases:
        print("dataset",dataset)
        if int(dataset.split('_')[-1])>=50000:
                continue
            
        now_mb=iter[iter.dataset == dataset]
        # print(now_mb)
        # print(len(now_mb))
        now=now_mb[['size','dim','graph_type','NN','ef_construction','k']]
        target=now_mb[['recall']]
        print(now)
        now = torch.from_numpy(now.values).float()
        print(now)

dataset mnist6W_4000
          size  dim  graph_type  NN  ef_construction   k
4032  4.542178  784           0   2               70   1
4185  4.542178  784           0  25              140   1
4129  4.542178  784           0  15               60  10
4151  4.542178  784           0  15              130  30
4013  4.542178  784           0   1              110  30
4279  4.542178  784           0  45              120  10
3999  4.542178  784           0   1               70   1
4256  4.542178  784           0  40              150  30
4011  4.542178  784           0   1              110   1
4212  4.542178  784           0  30              120   1
4059  4.542178  784           0   5               50   1
tensor([[  4.5422, 784.0000,   0.0000,   2.0000,  70.0000,   1.0000],
        [  4.5422, 784.0000,   0.0000,  25.0000, 140.0000,   1.0000],
        [  4.5422, 784.0000,   0.0000,  15.0000,  60.0000,  10.0000],
        [  4.5422, 784.0000,   0.0000,  15.0000, 130.0000,  30.0000],
        [  4.54